In [1]:
import pandas as pd
import spacy
import en_core_web_sm
from collections import Counter

In [2]:
# df = pd.read_csv("./downloads/reviews100.csv")
# df['comment'] = df['comment'].apply(lambda x: x.strip())
# #df = df.sort_values(['stars'], ascending=False).iloc[0:30] # get 30 top reviews 
# df = df.sample(frac=1).iloc[0:30] # get 30 top reviews 
# df.to_csv("30_reviews.csv", index=False) # save to csv for manual review
# df

In [42]:
df = pd.read_csv("./30_reviews.csv")
# df = df.iloc[0:2]
df

,stars,comment
0,1.0 out of 5 stars,"So I decided I wanted to get a paper shredder,..."
1,1.0 out of 5 stars,I order a lot from Amazon for convenience but ...
2,5.0 out of 5 stars,Pretty thrilled with this shredder right now !...
3,1.0 out of 5 stars,I have now been through 3 of these shredders. ...
4,5.0 out of 5 stars,"Update: After over a year, the shredder is sti..."
5,1.0 out of 5 stars,"SAYS it's made for home office, I guess becau..."
6,5.0 out of 5 stars,Quiet machine with ample capacity both in numb...
7,1.0 out of 5 stars,Very disappointed. Received shredder and plugg...
8,5.0 out of 5 stars,The Shredder is listed as a light/medium duty ...
9,3.0 out of 5 stars,"Will only give it 3 stars, since it arrived br..."


In [47]:
# nlp = spacy.load("en_core_web_sm") # buggy on ipynb
nlp = en_core_web_sm.load() # for ipynb
noun_adj_pairs = {}
for comment in enumerate(df['comment']):
    doc = nlp(str(comment[1])) # change tuple to str for nlp
    for sent in doc.sents:
        sent_len = len(sent)
        #=== noun/pnoun - adj pair
        for i, token in enumerate(sent): #token has TEXT, LEMMA, POS, TAG, DEP, SHAPE, ALPHA, STOP
            if token.pos_ not in ('NOUN', 'PROPN'):
                continue
            for j in range(i+1, sent_len): # check for pairs after identified noun/pnoun
                if sent[j].pos_ == 'ADJ':
                    pair = (str(token), str(sent[j])) # change to str. ele in dict keys has to be hashable even inside tuples
                    # check if already exist in dictionary
                    if pair in noun_adj_pairs:
                        noun_adj_pairs[pair] += 1
                    else:
                        noun_adj_pairs[pair] = 1
        #=== noun/pnoun phrase - adj pair
        for _, chunk in enumerate(sent.noun_chunks):
            index_after_chunk = chunk.end
            if index_after_chunk >= sent_len:
                continue
            for k in range(index_after_chunk, sent_len):
                if sent[k].pos_ == 'ADJ':
                    pair = (str(chunk.text), str(sent[k])) # change to str. ele in dict keys has to be hashable even inside tuples
                    # check if already exist in dictionary
                    if pair in noun_adj_pairs:
                        noun_adj_pairs[pair] += 1
                    else:
                        noun_adj_pairs[pair] = 1
top_10_noun_adj_pairs = dict(Counter(noun_adj_pairs).most_common(5))
top_10_noun_adj_pairs
# top_10_noun_adj_pairs = {key: val for key, val in sorted(noun_adj_pairs.items(), key=lambda item: item[1])}
# print(noun_adj_pairs.keys())
# noun_adj_pairs = dict(sorted(noun_adj_pairs.items()))
# print(noun_adj_pairs)

{('one', 'same'): 6,
 ('Amazon', 'old'): 6,
 ('shredder', 'less'): 5,
 ('April', 'original'): 4,
 ('bags', 'old'): 4}